## Listas de Precios

Usando las herramientas que hemos visto en el curso podemos implementar una metodología que nos permita, a partir de una lista de productos de despensa; Encontrar los precios más económicos para cuatro superficies como supermercados o tiendas.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Extraemos la información para las distintas superficies en los cuales contamos con productos de Almacenes exito, Jumbo, Ara y D1.

In [9]:
#@ title Productos Exito
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,149):
  url = 'https://losprecios.co/%C3%A9xito_t6/despensa?p={0}'
  response = requests.get(url.format(i))

  soup = BeautifulSoup(response.text, 'html.parser')
  conts = soup.find_all(class_='bq-cn-r')
  for cont in conts:
    cont_text = cont.find(class_='mrg-0')
    # print(cont_text)
    cont_nombre = cont_text.find(class_='eít-enl-p')
    nombre = cont_nombre.text
    marca = cont_nombre.find_next('span').find_next('span').text
    presentacion = cont.find(class_='no-lín').text
    precio = cont.find(class_='t-ed-pr').text
    nombres.append(nombre)
    marcas.append(marca)
    presentaciones.append(presentacion)
    precios.append(precio)
    lugar.append('éxito')

df1 = pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})
df1.head()

,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche Dietética,Alquería,225 g,2 656 $,éxito
1,Crema de Leche,Alquería,225 g,2 528 $,éxito
2,Crema de Leche,Nestlé,186 g,4 680 $,éxito
3,Crema de Leche,Alquería,200 g,2 200 $,éxito
4,Crema de Leche,Nestlé,150 g,1 958 $,éxito


In [10]:
#@title productos Jumbo
url = 'https://losprecios.co/jumbo_t7/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,52):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('Jumbo')

df2 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})
df2

,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche,Parmalat,295 g,6 790 $,Jumbo
1,Crema de Leche Dietética,Alpina,210 g,3 690 $,Jumbo
2,Crema de Leche,Alpina,210 g,3 590 $,Jumbo
3,Crema de Leche,Nestlé,295 g,4 472 $,Jumbo
4,Crema de Leche,Nestlé,186 g,3 992 $,Jumbo
...,...,...,...,...,...
504,Almidón de Yuca,Maizena,300 g,8 590 $,Jumbo
505,Endulzante de Fructosa,del Éxito,450 g,16 990 $,Jumbo
506,Salsa Búfalo,Amazon,355 g,5 992 $,Jumbo
507,Base para Salsa de Champiñones,Knorr,33 g,2 690 $,Jumbo


In [11]:
#@title Productos D1
url = 'https://losprecios.co/d1_t1/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,19):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('D1')

df3 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})

df3.head()

,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche,Latti,200 g,1 190 $,D1
1,Aceite Vegetal,Gota de Oro,1 000 ml,9 590 $,D1
2,Aceite de Girasol,Don Olio,1 000 ml,5 800 $,D1
3,Aceite de Oliva Extra Virgen,Castell de Ferro,500 ml,16 990 $,D1
4,Aceite de Canola,Don Olio,1 000 ml,14 850 $,D1


In [172]:
#@title Productos Ara
url = 'https://losprecios.co/ara_t2/despensa?p={0}'
nombres=[]
marcas=[]
presentaciones = []
precios =[]
lugar = []
for i in range(1,14):
    response = requests.get(url.format(i))
    soup = BeautifulSoup(response.text, 'html.parser')

    conts = soup.find_all(class_ = 'bq-cn-r')
    for cont in conts:
        cont_text = cont.find(class_='mrg-0')
        # print(cont_text)
        cont_nombre = cont_text.find(class_='eít-enl-p')
        nombre = cont_nombre.text
        marca = cont_nombre.find_next('span').find_next('span').text
        presentacion = cont.find(class_='no-lín').text
        precio = cont.find(class_='t-ed-pr').text
        nombres.append(nombre)
        marcas.append(marca)
        presentaciones.append(presentacion)
        precios.append(precio)
        lugar.append('Ara')

df4 =  pd.DataFrame({
'nombres':nombres,
'marcas':marcas,
'presentaciones':presentaciones,
'precios':precios,
'lugar':lugar
})

df4.head()


,nombres,marcas,presentaciones,precios,lugar
0,Crema de Leche,D'lish,200 g,2 100 $,Ara
1,Crema de Leche,Alquería,200 g,2 100 $,Ara
2,Aceite de Canola,Olisun,900 ml,5 750 $,Ara
3,Aceite Vegetal,Sonelo,3 000 ml,11 400 $,Ara
4,Aceite de Girasol,Olisun,3 000 ml,25 950 $,Ara


Creamos una función auxiliar para extraer los tipos de productos

In [18]:
#@title Función Extracción de Tipos
import spacy
import nltk
from nltk.tokenize import word_tokenize
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
def extraer_tipo_producto(texto):
    palabras = word_tokenize(texto, language='spanish')
    etiquetado = nltk.pos_tag(palabras)
    for palabra, etiqueta in etiquetado:
        if etiqueta in ['NN', 'NNS', 'NNP', 'NNPS']:  # Sustantivos
            return palabra
    return None
text = df4["nombres"][1]
extraer_tipo_producto(text)

'Crema'

Implementamos la función y realizamos Limpieza de los datos

In [26]:
#
df_precios = pd.concat([df1,df2,df3,df4])
df_indxed = df_precios.reset_index().drop('index',axis=1)

df_typed = df_indxed.copy()
df_typed["tipo"] = [extraer_tipo_producto(word) for word in df_indxed["nombres"]]
df_typed["precios"] = df_typed["precios"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("$")[0]).astype(float)
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("g")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("ml")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: x.split("unidades")[0])
# df_typed["presentaciones"] = df_typed["presentaciones"].replace(["\u202f"," "],"", regex=True).apply(lambda x: float(x.split("k")[0]))
presentaciones = []
unidades = []
for i in df_typed["presentaciones"]:
    if i.__contains__('kg'):
        i = i.replace('kg','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        i = float(i)*1000
        unidades.append("g")
        presentaciones.append(i)
    elif i.__contains__('ml'):
        i = i.replace('ml','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("ml")
        presentaciones.append(i)
    elif i.__contains__('unidades'):
        i = i.replace('unidades','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("unidades")
        presentaciones.append(i)
    elif i.__contains__('g'):
        i = i.replace('g','')
        i = i.replace("\u202f","")
        i = i.replace(" ","")
        unidades.append("g")
        presentaciones.append(i)

df_typed["presentaciones"] = presentaciones
df_typed["unidades"] = unidades


In [27]:
df_typed["marcas"][df_indxed["marcas"].str.contains('del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('del ')].str.replace('del ','')
df_typed["marcas"][df_indxed["marcas"].str.contains('Del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('Del ')].str.replace('Del ','')

/tmp/ipykernel_98163/1156242320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_typed["marcas"][df_indxed["marcas"].str.contains('del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('del ')].str.replace('del ','')
/tmp/ipykernel_98163/1156242320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_typed["marcas"][df_indxed["marcas"].str.contains('Del ')] = df_typed["marcas"][df_indxed["marcas"].str.contains('Del ')].str.replace('Del ','')


In [120]:
df_typed["marcas"].unique().__len__()
data = df_typed.copy()
df_typed

,nombres,marcas,presentaciones,precios,lugar,tipo,unidades
0,Crema de Leche Dietética,Alquería,225,2656.0,éxito,Crema,g
1,Crema de Leche,Alquería,225,2528.0,éxito,Crema,g
2,Crema de Leche,Nestlé,186,4680.0,éxito,Crema,g
3,Crema de Leche,Alquería,200,2200.0,éxito,Crema,g
4,Crema de Leche,Nestlé,150,1958.0,éxito,Crema,g
...,...,...,...,...,...,...,...
2286,Maíz Trillado Blanco,Costal,1000,1700.0,Ara,Maíz,g
2287,Base para Salsa Bechamel,Minotto,50,1950.0,Ara,Base,g
2288,Mezcla para Arepas de Choclo,Pan,1000,5300.0,Ara,Mezcla,g
2289,Pasta en Letras,Doria,250,1450.0,Ara,Pasta,g


In [163]:
# Definiendo una lista de ejemplo de productos de despensa
lista_productos_expandida = [
    {'tipo': 'Crema', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Aceite', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Azúcar', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Arroz', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Mantequilla', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Margarina', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Huevos', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Queso', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Atún', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Espaguetis', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Salsa', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Sal', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Cristales', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Harina', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Chocolate', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Lentejas', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Panela', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Mayonesa', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Avena', 'presentacion': 500, 'unidades': 'g'},
    {'tipo': 'Cereal', 'presentacion': 500, 'unidades': 'g'}
]
data = df_typed.copy()
# Función para encontrar la opción más barata para un producto dado
def encontrar_opcion_mas_barata(tipo_producto, presentacion_deseada, unidad):
    # Filtrando el dataset para encontrar opciones del producto
    opciones_producto = data[(data['tipo'] == tipo_producto) & (data['unidades'] == unidad)]

    # Verificando si existe la presentación exacta
    presentacion_exacta = opciones_producto[opciones_producto['presentaciones'] == presentacion_deseada]
    if not presentacion_exacta.empty:
        producto_mas_barato = presentacion_exacta.loc[presentacion_exacta['precios'].idxmin()]
        return producto_mas_barato, producto_mas_barato['precios']

    # Si no, buscando opciones con presentaciones más pequeñas
    else:
        opciones_posibles = opciones_producto[opciones_producto['presentaciones'] < presentacion_deseada]
        if opciones_posibles.empty:
            return None, None

        # Calculando el costo de comprar múltiples unidades para alcanzar la presentación deseada
        opciones_posibles['costo_total'] = opciones_posibles['precios'] * (presentacion_deseada / opciones_posibles['presentaciones'])
        opcion_mas_barata = opciones_posibles.loc[opciones_posibles['costo_total'].idxmin()]

        return opcion_mas_barata, opcion_mas_barata['costo_total']

# Creando una lista para almacenar los resultados
resultados_optimizacion = []
tipos_columnas = data.dtypes

# Corrigiendo el tipo de datos de la columna 'presentaciones' si es necesario
if tipos_columnas['presentaciones'] != 'float64' and tipos_columnas['presentaciones'] != 'int64':
    data['presentaciones'] = pd.to_numeric(data['presentaciones'], errors='coerce')

# Reintentando el cálculo de la opción más barata
resultados_optimizacion = []

for producto in lista_productos_expandida:
    resultado, costo = encontrar_opcion_mas_barata(producto['tipo'], producto['presentacion'], producto['unidades'])
    if resultado is not None:
        resultados_optimizacion.append({
            "tipo": producto['tipo'],
            "presentacion": producto['presentacion'],
            "unidades": producto['unidades'],
            "marca": resultado['marcas'],
            "precio": costo,
            "lugar": resultado['lugar']
        })

# Convirtiendo los resultados en un DataFrame para una mejor visualización
resultados_optimizacion_df = pd.DataFrame(resultados_optimizacion)
resultados_optimizacion_df


/tmp/ipykernel_98163/4053151023.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opciones_posibles['costo_total'] = opciones_posibles['precios'] * (presentacion_deseada / opciones_posibles['presentaciones'])
/tmp/ipykernel_98163/4053151023.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opciones_posibles['costo_total'] = opciones_posibles['precios'] * (presentacion_deseada / opciones_posibles['presentaciones'])


,tipo,presentacion,unidades,marca,precio,lugar
0,Crema,500,g,Colanta,12100.000000,éxito
1,Aceite,500,g,Don Olio,18092.105263,D1
2,Azúcar,500,g,Éxito,1400.000000,éxito
3,Arroz,500,g,Diana,1800.000000,Ara
4,Mantequilla,500,g,Ekono,6380.000000,éxito
5,Margarina,500,g,Éxito,5510.000000,éxito
6,Queso,500,g,Alpina,5368.421053,éxito
7,Atún,500,g,Ekono,7117.647059,éxito
8,Espaguetis,500,g,Doria,100.000000,Ara
9,Salsa,500,g,Ekono,2490.000000,éxito


In [128]:
# Filtrando productos cuyas presentaciones están en gramos
productos_en_gramos = data[data['unidades'] == 'g']

# Seleccionando 30 productos únicos basados en sus tipos
productos_seleccionados = productos_en_gramos.drop_duplicates(subset=['tipo']).head(30)

# Creando una lista de mercado ficticia
# Multiplicando algunos productos por cantidades ficticias
# Para este ejemplo, multiplicaremos cada precio por un factor entre 1 y 5

import numpy as np

np.random.seed(0)  # Para reproducibilidad
cantidades = np.random.randint(1, 6, size=len(productos_seleccionados))  # Generando cantidades aleatorias entre 1 y 5
productos_seleccionados['cantidad'] = cantidades
productos_seleccionados['precio_total'] = productos_seleccionados['precios'] * productos_seleccionados['cantidad']

productos_seleccionados[['tipo', 'presentaciones', 'cantidad']]


,tipo,presentaciones,cantidad
0,Crema,225.0,5
93,Azúcar,1000.0,1
136,Arroz,2500.0,4
157,Mantequilla,500.0,4
175,Margarina,500.0,4
223,Queso,125.0,2
234,Atún,350.0,4
270,Espaguetis,500.0,3
318,Salsa,380.0,5
356,Sal,465.0,1


In [133]:
data

,nombres,marcas,presentaciones,precios,lugar,tipo,unidades
0,Crema de Leche Dietética,Alquería,225.0,2656.0,éxito,Crema,g
1,Crema de Leche,Alquería,225.0,2528.0,éxito,Crema,g
2,Crema de Leche,Nestlé,186.0,4680.0,éxito,Crema,g
3,Crema de Leche,Alquería,200.0,2200.0,éxito,Crema,g
4,Crema de Leche,Nestlé,150.0,1958.0,éxito,Crema,g
...,...,...,...,...,...,...,...
2286,Maíz Trillado Blanco,Costal,1000.0,1700.0,Ara,Maíz,g
2287,Base para Salsa Bechamel,Minotto,50.0,1950.0,Ara,Base,g
2288,Mezcla para Arepas de Choclo,Pan,1000.0,5300.0,Ara,Mezcla,g
2289,Pasta en Letras,Doria,250.0,1450.0,Ara,Pasta,g


In [135]:
# Verificando si el conjunto de datos actual proporciona información sobre precios en diferentes almacenes
almacenes_precios = data[['tipo', 'marcas', 'precios', 'lugar']].drop_duplicates()
almacenes_precios.head()

# Filtrando los productos seleccionados en diferentes almacenes y sus precios
productos_filtrados = almacenes_precios[almacenes_precios['tipo'].isin(productos_seleccionados['tipo'])]

# Debido a que algunos productos pueden tener el mismo nombre pero ser de marcas diferentes,
# también incluiremos la marca en el filtro
productos_filtrados = productos_filtrados[productos_filtrados['marcas'].isin(productos_seleccionados['marcas'])]

productos_filtrados  # Mostrando los primeros 10 registros para verificar la información obtenida


,tipo,marcas,precios,lugar
0,Crema,Alquería,2656.0,éxito
1,Crema,Alquería,2528.0,éxito
3,Crema,Alquería,2200.0,éxito
13,Crema,Alquería,2940.0,éxito
15,Crema,Alquería,16838.0,éxito
...,...,...,...,...
2162,Crema,Alquería,2100.0,Ara
2185,Margarina,Rama,4800.0,Ara
2201,Sal,Refisal,2400.0,Ara
2202,Sal,Refisal,1000.0,Ara


In [138]:
productos_filtrados["tipo"].unique()

array(['Crema', 'Aceite', 'Azúcar', 'Arroz', 'Mantequilla', 'Margarina',
       'Queso', 'Atún', 'Espaguetis', 'Salsa', 'Sal', 'Cristales',
       'Harina', 'Chocolate', 'Lentejas', 'Panela', 'Mayonesa', 'Avena',
       'Cereal', 'Hojuelas', 'Muesli', 'Granola', 'Maíz', 'Fríjoles',
       'Miel', 'Garbanzos', 'Gelatina', 'Polvo', 'Guacamole', 'Pasta'],
      dtype=object)

In [164]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

# Creando el modelo de programación lineal
modelo = LpProblem("Optimizacion_de_Compras", LpMinimize)

# Lista de almacenes y productos únicos
almacenes = list(productos_filtrados['lugar'].unique())
productos = list(productos_filtrados['tipo'].unique())


In [171]:
costo_productos

14590.0*Cantidad_('Aceite',_'Jumbo') + 14050.0*Cantidad_('Aceite',_'éxito') + 8810.0*Cantidad_('Arroz',_'éxito') + 11390.0*Cantidad_('Atún',_'Jumbo') + 10640.0*Cantidad_('Atún',_'éxito') + 8990.0*Cantidad_('Avena',_'éxito') + 3490.0*Cantidad_('Azúcar',_'Jumbo') + 3170.0*Cantidad_('Azúcar',_'éxito') + 7990.0*Cantidad_('Cereal',_'D1') + 9790.0*Cantidad_('Cereal',_'Jumbo') + 20050.0*Cantidad_('Cereal',_'éxito') + 13390.0*Cantidad_('Chocolate',_'Jumbo') + 13600.0*Cantidad_('Chocolate',_'éxito') + 2100.0*Cantidad_('Crema',_'Ara') + 12390.0*Cantidad_('Crema',_'Jumbo') + 2656.0*Cantidad_('Crema',_'éxito') + 8590.0*Cantidad_('Cristales',_'Jumbo') + 8430.0*Cantidad_('Cristales',_'éxito') + 4390.0*Cantidad_('Espaguetis',_'Jumbo') + 2790.0*Cantidad_('Espaguetis',_'éxito') + 4890.0*Cantidad_('Fríjoles',_'éxito') + 5280.0*Cantidad_('Garbanzos',_'éxito') + 2350.0*Cantidad_('Gelatina',_'Jumbo') + 2280.0*Cantidad_('Gelatina',_'éxito') + 10210.0*Cantidad_('Granola',_'éxito') + 6690.0*Cantidad_('Guacamo

In [152]:

# Variables de decisión: cantidad a comprar de cada producto en cada almacén
cantidad_vars = LpVariable.dicts("Cantidad", [(i,j) for i in productos for j in almacenes], lowBound=0, cat='Integer')

# Variable de decisión para determinar si se visita un almacén o no (0 o 1)
visita_vars = LpVariable.dicts("Visita", almacenes, cat='Binary')

# Función objetivo: Minimizar el costo total (precio de productos + costo de desplazamiento)
costo_productos = lpSum([cantidad_vars[i,j] * productos_filtrados[(productos_filtrados['tipo'] == i) & 
                                                                  (productos_filtrados['lugar'] == j)]['precios'].values[0]
                         if len(productos_filtrados[(productos_filtrados['tipo'] == i) & 
                                                    (productos_filtrados['lugar'] == j)]['precios'].values) > 0
                         else 0
                         for i in productos for j in almacenes])

costo_desplazamiento = lpSum([visita_vars[j] * 20000 for j in almacenes])

modelo += costo_productos + costo_desplazamiento

# Restricción de presupuesto
modelo += costo_productos + costo_desplazamiento <= 600000

# Solucionando el modelo
modelo.solve()

# Estado del modelo y valores de las variables de decisión si es exitoso
estado_modelo = LpStatus[modelo.status]

estado_modelo


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/dancastarod/.local/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d497bb8e85844d78b8ebeb181f503bdc-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/d497bb8e85844d78b8ebeb181f503bdc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 251 RHS
At line 253 BOUNDS
At line 315 ENDATA
Problem MODEL has 1 rows, 61 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts o

'Optimal'

In [161]:
{(i,j): cantidad_vars[i,j].varValue for i in productos for j in almacenes }


{('Crema', 'éxito'): 0.0,
 ('Crema', 'Jumbo'): 0.0,
 ('Crema', 'D1'): None,
 ('Crema', 'Ara'): 0.0,
 ('Aceite', 'éxito'): 0.0,
 ('Aceite', 'Jumbo'): 0.0,
 ('Aceite', 'D1'): None,
 ('Aceite', 'Ara'): None,
 ('Azúcar', 'éxito'): 0.0,
 ('Azúcar', 'Jumbo'): 0.0,
 ('Azúcar', 'D1'): None,
 ('Azúcar', 'Ara'): None,
 ('Arroz', 'éxito'): 0.0,
 ('Arroz', 'Jumbo'): None,
 ('Arroz', 'D1'): None,
 ('Arroz', 'Ara'): None,
 ('Mantequilla', 'éxito'): 0.0,
 ('Mantequilla', 'Jumbo'): 0.0,
 ('Mantequilla', 'D1'): None,
 ('Mantequilla', 'Ara'): None,
 ('Margarina', 'éxito'): 0.0,
 ('Margarina', 'Jumbo'): 0.0,
 ('Margarina', 'D1'): 0.0,
 ('Margarina', 'Ara'): 0.0,
 ('Queso', 'éxito'): 0.0,
 ('Queso', 'Jumbo'): None,
 ('Queso', 'D1'): None,
 ('Queso', 'Ara'): None,
 ('Atún', 'éxito'): 0.0,
 ('Atún', 'Jumbo'): 0.0,
 ('Atún', 'D1'): None,
 ('Atún', 'Ara'): None,
 ('Espaguetis', 'éxito'): 0.0,
 ('Espaguetis', 'Jumbo'): 0.0,
 ('Espaguetis', 'D1'): None,
 ('Espaguetis', 'Ara'): None,
 ('Salsa', 'éxito'): 0.0,
 (